# 🔧 Werkplek Inspectie AI - YOLOv8 Classification Training

Training notebook voor Google Colab - **Classification Mode**

**Updates:**
- ✅ Data Augmentation (Rotatie 90°, Flip, Zoom) toegevoegd
- ✅ ImageFolder structuur (automatische class handling)
- ✅ Support voor hertrainen (Fine-tuning)

**⚠️ BELANGRIJK: Zet Runtime op GPU!**
- Runtime → Change runtime type → GPU (T4)
- Zonder GPU duurt het trainen uren!

## 1️⃣ Setup Omgeving

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Installeer dependencies
!pip install ultralytics opencv-python pillow -q

In [ ]:
# Imports
import os
import shutil
from pathlib import Path
import random
from ultralytics import YOLO
import torch

print(f"✅ PyTorch versie: {torch.__version__}")
print(f"✅ CUDA beschikbaar: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

## 2️⃣ Upload Dataset

1. Upload **MASTER_DATASET.zip** naar de Colab bestandslijst (links).
2. Draai de code hieronder om hem uit te pakken.

In [ ]:
# 🔹 STAP 2A: Upload je MASTER_DATASET.zip
# Sleep MASTER_DATASET.zip naar de bestandslijst links!

import os
import shutil
import zipfile
from pathlib import Path

# Configuratie
ZIP_PATH = '/content/MASTER_DATASET.zip'
EXTRACT_PATH = '/content/temp_extract'
DATASET_RAW = Path('/content/dataset_raw')

if not os.path.exists(ZIP_PATH):
    print("❌ ERROR: Geen MASTER_DATASET.zip gevonden! Upload deze eerst.")
else:
    # 1. Schoon begin
    if DATASET_RAW.exists():
        shutil.rmtree(DATASET_RAW)
    DATASET_RAW.mkdir(parents=True)

    # 2. Unzip
    print("🚀 Uitpakken dataset...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_PATH)

    # 3. Sorteer (Flatten structuur)
    # Soms zit het in een submap, dit trekt alles recht
    print("📂 Organiseren van mappen...")
    count = 0
    for file_path in Path(EXTRACT_PATH).rglob('*'):
        if file_path.is_file() and file_path.suffix.lower() in ['.jpg', '.jpeg', '.png']:
            # De mapnaam waar de file in zit, is de klassenaam (bijv 'OK')
            class_name = file_path.parent.name
            
            # Maak doelmap
            dest_folder = DATASET_RAW / class_name
            dest_folder.mkdir(parents=True, exist_ok=True)
            
            # Kopieer
            shutil.copy(file_path, dest_folder / f"{count}_{file_path.name}")
            count += 1
            
    # Opruimen temp
    shutil.rmtree(EXTRACT_PATH)
    print(f"✅ Klaar! {count} afbeeldingen klaar voor training in {DATASET_RAW}")

In [ ]:
# Dataset configuratie
RAW_DATA_DIR = Path("/content/dataset_raw")
OUTPUT_DIR = Path("/content/yolo_dataset")
TRAIN_SPLIT = 0.8

# DEFINIEER HIER JE CLASSES EN VOLGORDE
CLASS_MAPPING = {
    "OK": 0,
    "NOK alles weg": 1,
    "NOK hamer weg": 2,
    "NOK schaar weg": 3,
    "NOK schaar en sleutel weg": 4,
    "NOK sleutel weg": 5,
    "NOK schaar en hamer weg": 6,
    "NOK hamer en sleutel weg": 7
}

CLASS_NAMES = [
    "ok",
    "nok_alles_weg",
    "nok_hamer_weg",
    "nok_schaar_weg",
    "nok_schaar_sleutel_weg",
    "nok_sleutel_weg",
    "nok_alleen_sleutel",
    "nok_alleen_schaar"
]

In [ ]:
# Prepareer dataset - ImageFolder format
def create_yolo_dataset():
    """Converteer naar YOLO ImageFolder format"""

    # Verwijder oude output
    if OUTPUT_DIR.exists():
        shutil.rmtree(OUTPUT_DIR)

    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

    # Maak class folders: 0_ok, 1_nok_alles_weg, etc.
    for split in ['train', 'val']:
        for idx, name in enumerate(CLASS_NAMES):
            folder_name = f"{idx}_{name}"
            (OUTPUT_DIR / split / folder_name).mkdir(parents=True, exist_ok=True)

    print("✅ Directory structuur (ImageFolder)\n")

    # Verzamel images
    all_images = []
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}

    for folder_name, class_id in CLASS_MAPPING.items():
        possible_paths = [
            RAW_DATA_DIR / folder_name,
            RAW_DATA_DIR / "AI afbeeldingen" / folder_name,
        ]

        folder_path = None
        for path in possible_paths:
            if path.exists():
                folder_path = path
                break

        if not folder_path:
            print(f"⚠️  Folder niet gevonden: {folder_name}")
            continue

        images = [f for f in folder_path.glob('*') if f.suffix.lower() in image_extensions]
        print(f"✅ {folder_name}: {len(images)} afbeeldingen (class {class_id})")

        for img_path in images:
            all_images.append((img_path, class_id))

    # Shuffle en split
    random.seed(42)
    random.shuffle(all_images)

    split_idx = int(len(all_images) * TRAIN_SPLIT)
    train_images = all_images[:split_idx]
    val_images = all_images[split_idx:]

    print(f"\n✅ Split: {len(train_images)} train, {len(val_images)} val\n")

    # Kopieer naar class folders
    for split_name, image_list in [('train', train_images), ('val', val_images)]:
        for idx, (img_path, class_id) in enumerate(image_list):
            class_name = CLASS_NAMES[class_id]
            folder_name = f"{class_id}_{class_name}"
            new_name = f"{split_name}_{idx}_{img_path.name}"
            dst_img = OUTPUT_DIR / split_name / folder_name / new_name
            shutil.copy2(img_path, dst_img)

    print("✅ Dataset klaar! (ImageFolder format)")
    return OUTPUT_DIR

# Run
print("🚀 Start preprocessing...\n")
dataset_path = create_yolo_dataset()
print(f"\n✅ Dataset pad: {dataset_path}")

## 4️⃣ Train YOLOv8 Model

In [ ]:
# Training Configuratie
EPOCHS = 100
BATCH_SIZE = 16
IMAGE_SIZE = 640
MODEL_SIZE = 'n'  # 'n' (nano), 's' (small)

# HERTRAINING OPTIE
USE_PRETRAINED = False # Zet op True om verder te trainen met een oud model
PRETRAINED_MODEL_PATH = '/content/drive/MyDrive/werkplek_classifier.pt'

print("🎯 Training Configuratie")

In [ ]:
# Laad Model
if USE_PRETRAINED and os.path.exists(PRETRAINED_MODEL_PATH):
    print(f"🔄 Laden van bestaand model: {PRETRAINED_MODEL_PATH}")
    model = YOLO(PRETRAINED_MODEL_PATH)
else:
    print(f"🆕 Start training van scratch (yolov8{MODEL_SIZE}-cls.pt)")
    model = YOLO(f'yolov8{MODEL_SIZE}-cls.pt')

In [ ]:
# START TRAINING! 🚀
results = model.train(
    data='/content/yolo_dataset',
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=IMAGE_SIZE,
    device=0,
    project='runs/classify',
    name='werkplek_inspect',
    exist_ok=True,
    patience=20,
    save=True,

    # DATA AUGMENTATION (Jouw updates!)
    degrees=90.0,      # Rotatie
    translate=0.1,     # Positie verschuiving
    scale=0.5,         # Zoom in/out
    fliplr=0.5,        # Horizontale flip
    flipud=0.5,        # Verticale flip
    hsv_h=0.015,       # Kleur variatie
    hsv_s=0.7,         # Saturatie
    hsv_v=0.4          # Helderheid
)

## 5️⃣ Download Model

In [ ]:
# Download
from google.colab import files

!cp runs/classify/werkplek_inspect/weights/best.pt /content/werkplek_classifier.pt
files.download('/content/werkplek_classifier.pt')